In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial import distance
from exp_nume_ex import lattice
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

In [41]:
E = 5
Col = 2
K = Col * Col
int_Col = 2
int_K = int_Col * int_Col
M = 1.0
N = 1.0
Scaling = 10.0 / int_Col
alter_T_num = 0.5
S_total = 100
S_bar = S_total / int_K
t = 0.1
alpha_1 = 0.4
alpha_2 = 0.4
beta_1 = 0.4
beta_2 = 0.4
epsilon = 1e-3

city_network = lattice.make_lattice(Col)

Coordinate_Data = \
np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
           city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

n = np.full((int_K, int_K), N / (int_K * int_K))

# T行列の計算
T = np.array([[max(Scaling * t * alter_T_num, t * distance_matrix[i][j]) 
              for j in range(int_K)] for i in range(int_K)])

# mベクトルの作成
m = np.full(int_K, M / int_K)

In [42]:
# --- 目的関数と勾配の定義 ---

# def objective(x):
#     """目的関数 Z_{SD}"""
#     R = x[:K]
#     W = x[K:]
    
#     if np.any(R < 0) or np.any(W < 0):
#         R += 1e-3
#         W += 1e-3
#         print("Warning: Negative values detected in R or W")
    
#     term1 = np.sum((1 - beta_1 - beta_2) * (beta_1 / R)**(beta_1 / (1 - beta_1 - beta_2)) * (beta_2 / W)**(beta_2 / (1 - beta_1 - beta_2)) * m)
#     term2 = np.sum(S_bar * R)
#     term3 = 0
#     for i in range(K):
#         for j in range(K):
#             term3 += (E * W[j] + (1 - alpha_1 - alpha_2) * (1 / np.exp(T[i, j]))**(1 / (1 - alpha_1 - alpha_2)) * \
#                       (alpha_1 / R[i])**(alpha_1 / (1 - alpha_1 - alpha_2)) * (alpha_2 / W[j])**(alpha_2 / (1 - alpha_1 - alpha_2))) * n[i, j]
#     return term1 + term2 + term3

# --- 目的関数 (COBYLA は勾配不要, epsilon を導入) ---
def objective(x):
    """目的関数 Z_{SD}"""
    R = x[:K]
    W = x[K:]
    
    R = np.maximum(R, 1e-5)
    W = np.maximum(W, 1e-5)
    
    # print("R_min:", np.min(R))
    # print("W_min:", np.min(W))

    term1 = np.sum((1 - beta_1 - beta_2) * (beta_1 / R)**(beta_1 / (1 - beta_1 - beta_2)) * (beta_2 / W)**(beta_2 / (1 - beta_1 - beta_2)) * m)
    term2 = np.sum(S_bar * R)
    term3 = 0
    for i in range(K):
        for j in range(K):
            term3 += (E * W[j] + (1 - alpha_1 - alpha_2) * np.exp(-T[i, j]) * \
                      (alpha_1 / R[i])**(alpha_1 / (1 - alpha_1 - alpha_2)) * (alpha_2 / W[j])**(alpha_2 / (1 - alpha_1 - alpha_2))) * n[i, j]
    return term1 + term2 + term3

def gradient(x):
    """勾配ベクトル (∂Z_{SD}/∂R, ∂Z_{SD}/∂W)"""
    R = x[:K]
    W = x[K:]
    
    R = np.maximum(R, 1e-5)
    W = np.maximum(W, 1e-5)
    
    grad_R = np.zeros(K)
    grad_W = np.zeros(K)

    for i in range(K):
        sum_term = 0
        for j in range(K):
            sum_term += (1 / np.exp(T[i, j]))**(1 / (1 - alpha_1 - alpha_2)) * \
                         (alpha_1 / R[i])**((1 - alpha_2) / (1 - alpha_1 - alpha_2)) * \
                         (alpha_2 / W[j])**(alpha_2 / (1 - alpha_1 - alpha_2)) * n[i, j]
        grad_R[i] = S_bar - sum_term - (beta_1 / R[i])**((1 - beta_2) / (1 - beta_1 - beta_2)) * (beta_2 / W[i])**(beta_2 / (1 - beta_1 - beta_2)) * m[i]

    for j in range(K):
        sum_term1 = 0
        sum_term2 = 0
        for i in range(K):
          sum_term1 += (1 / np.exp(T[i, j]))**(1 / (1 - alpha_1 - alpha_2)) * (alpha_1 / R[i])**(alpha_1 / (1 - alpha_1 - alpha_2)) * (alpha_2 / W[j])**((1-alpha_1) / (1 - alpha_1 - alpha_2)) * n[i, j]
          sum_term2 += (beta_1 / R[i])**(beta_1 / (1 - beta_1 - beta_2)) * (beta_2 / W[i])**((1 - beta_1) / (1 - beta_1 - beta_2)) * m[j]
        grad_W[j] = E * np.sum(n[:, j]) - sum_term1 - sum_term2

    return np.concatenate((grad_R, grad_W))

In [43]:
# --- 制約条件 (各要素 >= 1e-3) ---
# def constraint(x):
#     # return x - 1e-3  # 各要素から 1e-3 を引いたものを返す
#     return x# 各要素から 1e-3 を引いたものを返す

# --- 最適化の実行 ---

# 初期値
x0 = np.ones(2 * K)

# 制約条件 (リスト形式)
cons = ({'type': 'ineq', 'fun': constraint})

# 最適化 (COBYLA)
result = minimize(objective, x0, method='COBYLA', constraints=None,
                  options={'disp': True, 'rhobeg': 0.5, 'maxiter': 200000, 'tol': 1e-9})

# --- 結果の表示 ---

print("最適化結果:")
print(result)

# 最適解
R_optimal = result.x[:K]
W_optimal = result.x[K:]

print("\nOptimal R:")
print(R_optimal)
print("\nOptimal W:")
print(W_optimal)

最適化結果:
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 13.229915569277438
       x: [ 5.292e-02  5.292e-02  5.292e-02  5.292e-02  1.058e+00
            1.058e+00  1.058e+00  1.058e+00]
    nfev: 25761
   maxcv: 0.0

Optimal R:
[0.05291971 0.05291971 0.05291971 0.05291971]

Optimal W:
[1.05839186 1.05839181 1.05839186 1.05839189]


In [45]:
# 非負制約を LinearConstraint で定義
lb = np.full(2 * K, 1e-7)  # 下限
ub = np.full(2 * K, np.inf) # 上限
linear_constraint = LinearConstraint(np.eye(2 * K), lb, ub)


# --- 最適化の実行 ---

# 初期値を変更 (0.5 から 1.5 の範囲の一様乱数)
x0 = np.ones(2 * K)

# 最適化の実行 (trust-constr)
result = minimize(objective, x0, method='trust-constr', jac=gradient,
                constraints=[linear_constraint],
                options={'disp': True, 'verbose': 0, 'maxiter': 1000, 'gtol': 1e-8, 'xtol':1e-8})
# verboseレベルを上げ，gtolとxtolを調整


# --- 結果の表示 ---

print("最適化結果:")
print(result)

# 最適解
R_optimal = result.x[:K]
W_optimal = result.x[K:]

print("\nOptimal R:")
print(R_optimal)
print("\nOptimal W:")
print(W_optimal)

`xtol` termination condition is satisfied.
Number of iterations: 370, function evaluations: 439, CG iterations: 394, optimality: 9.28e-01, constraint violation: 0.00e+00, execution time: 0.33 s.
最適化結果:
           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: 13.240051050001853
                 x: [ 5.193e-02  5.217e-02  5.497e-02  5.198e-02  1.085e+00
                      1.084e+00  1.096e+00  1.083e+00]
               nit: 370
              nfev: 439
              njev: 439
              nhev: 0
          cg_niter: 394
      cg_stop_cond: 2
              grad: [ 7.612e+00  7.834e+00  1.063e+01  7.617e+00 -1.706e+00
                     -1.707e+00 -1.700e+00 -1.706e+00]
   lagrangian_grad: [ 2.048e-02  2.126e-02  3.201e-02  2.052e-02 -9.224e-01
                     -9.226e-01 -9.278e-01 -9.215e-01]
            constr: [array([ 5.193e-02,  5.217e-02,  5.497e-02,  5.198e-02,
                            1.085e+00,  

In [11]:
# --- 制約条件 ---
# 非負制約 + 各変数に下限を設定
bounds = [(1e-3, None) for _ in range(2 * K)]  # 下限を1e-9に設定

# --- 最適化の実行 ---
# 初期値を変更 (0.5 から 1.5 の範囲の一様乱数)
x0 = np.ones(2 * K)

# 最適化の実行
result = minimize(objective, x0, method='SLSQP',
                  jac=gradient, bounds=bounds, options={'disp': True, 'ftol':1e-8, 'maxiter': 10000})  #収束判定を厳しめに，最大反復回数も指定．

# --- 結果の表示 ---

print("最適化結果:")
print(result)

# 最適解
R_optimal = result.x[:K]
W_optimal = result.x[K:]

print("\nOptimal R:")
print(R_optimal)
print("\nOptimal W:")
print(W_optimal)

Iteration limit reached    (Exit mode 9)
            Current function value: 16.605722267639983
            Iterations: 10000
            Function evaluations: 109732
            Gradient evaluations: 9999
最適化結果:
 message: Iteration limit reached
 success: False
  status: 9
     fun: 16.605722267639983
       x: [ 2.858e-02  2.815e-02 ...  2.388e+00  2.303e+00]
     nit: 10000
     jac: [ 5.750e-01  6.017e-01 ... -6.325e-01 -6.314e-01]
    nfev: 109732
    njev: 9999

Optimal R:
[0.02857859 0.02814868 0.02844109 0.02843156 0.02824855 0.02791372
 0.02785613 0.02809028 0.02825507 0.02793164 0.02798475 0.02818941
 0.02869011 0.02856564 0.02834632 0.02854882]

Optimal W:
[2.3057304  2.40272726 2.42145133 2.33267314 2.38328148 2.4791817
 2.48931594 2.39957236 2.37649542 2.46941528 2.47431311 2.38757707
 2.30282057 2.38889165 2.38787    2.30268533]
